In [5]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
import time

service = Service(ChromeDriverManager().install())
options = Options()
# options.add_argument('--headless')
# options.add_argument('--disable-gpu')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--window-size=1920x1080')
options.add_argument('--start-maximized')
options.add_argument('--user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/139.0.0.0 Safari/537.36')
options.add_argument('--lang=ko_KR')

driver = webdriver.Chrome(service=service, options=options)

driver.get("https://www.musinsa.com/main/musinsa/recommend?gf=A")
time.sleep(3)

search_button = driver.find_element(By.CLASS_NAME, "sc-dzagxs-2")
search_button.click()
time.sleep(2)

popular_keywords = driver.find_elements(By.CSS_SELECTOR, ".search-home-popular-wrap .sc-1xzm8bl-2")
target_keyword = popular_keywords[1].text
print("목표키워드 :", target_keyword)

search_input = driver.find_element(By.CLASS_NAME, "search-home-search-bar-keyword")
search_input.clear()
search_input.send_keys(target_keyword)
time.sleep(1)

search_input.send_keys(Keys.RETURN)
time.sleep(5)

print("검색 완료. 현재 페이지 타이틀 :", driver.title)
driver.quit()

목표키워드 : 반팔
검색 완료. 현재 페이지 타이틀 : 반팔 | 무신사 추천 상품


In [9]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [14]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time
import re

service = Service(ChromeDriverManager().install())
options = Options()
# options.add_argument('--headless')
# options.add_argument('--disable-gpu')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--window-size=1920x1080')
options.add_argument('--start-maximized')
options.add_argument('--user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/139.0.0.0 Safari/537.36')
options.add_argument('--lang=ko_KR')

driver = webdriver.Chrome(service=service, options=options)

driver.get("https://www.musinsa.com/main/musinsa/recommend?gf=A")
time.sleep(3)

search_button = driver.find_element(By.CLASS_NAME, "sc-dzagxs-2")
search_button.click()
time.sleep(2)

popular_keywords = driver.find_elements(By.CSS_SELECTOR, ".search-home-popular-wrap .sc-1xzm8bl-2")
target_keyword = popular_keywords[1].text
print("목표키워드 :", target_keyword)

search_input = driver.find_element(By.CLASS_NAME, "search-home-search-bar-keyword")
search_input.clear()
search_input.send_keys(target_keyword)
time.sleep(1)

search_input.send_keys(Keys.RETURN)
time.sleep(5)

print("검색 완료. 현재 페이지 타이틀 :", driver.title)

scroll_pause = 2

for _ in range(10) :
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(scroll_pause)

products = driver.find_elements(By.CSS_SELECTOR, "div.sc-d36st-1.elXjMR")[:100]
product_data = []

for product in products :
    try :
        name = product.find_element(By.CSS_SELECTOR, "div.sc-cMuefe.jIlLeV a.gtm-select-item span").text.strip()
        url = product.find_element(By.CSS_SELECTOR, "div.sc-cMuefe.jIlLeV a.gtm-select-item").get_attribute("href")
        review_info = product.find_elements(By.CSS_SELECTOR, "span.text-etc_11px_reg.text-yellow.font-pretendard")[1].text
        review_count = int(re.sub(r"[^\d]", "", review_info)) if review_info else 0
        product_data.append({"상품명": name, "URL": url, "리뷰수": review_count})
    except :
        continue

df = pd.DataFrame(product_data)
df_top30 = df.sort_values(by="리뷰수", ascending=False).head(30).reset_index(drop=True)

driver.quit()
df_top30.head()

목표키워드 : 반팔
검색 완료. 현재 페이지 타이틀 : 반팔 | 무신사 추천 상품


,상품명,URL,리뷰수
0,W TWINKLE SELLER LOGO SS TEE WHITE(CV2EMFT517A),https://www.musinsa.com/products/3052406,652
1,레터링 아카이브 티셔츠 [화이트],https://www.musinsa.com/products/3820600,647
2,바이시클 오버사이즈 반팔 티셔츠 [그린],https://www.musinsa.com/products/2534493,532
3,냥이가 지켜보고 있다 티셔츠 [화이트],https://www.musinsa.com/products/3203072,502
4,트리플 나비 자수 오버핏 반팔 티셔츠 ( 챠콜 ),https://www.musinsa.com/products/3283586,500


In [11]:
pip install openpyxl

Note: you may need to restart the kernel to use updated packages.


In [16]:
from openpyxl import Workbook

driver = webdriver.Chrome(service=service, options=options)
all_reviews = []

for idx, row in df_top30.iterrows() :
    print(f"{idx + 1}번째 상품리뷰 수집 중 :", {row["상품명"]})
    driver.get(row["URL"])
    time.sleep(2)

    try :
        review_tab = driver.find_element(By.XPATH, "//button[@data-button-id='prd_review_tab']")
        review_tab.click()
        time.sleep(2)

        for _ in range(3) :
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(1.5)
        review_elements = driver.find_elements(By.CSS_SELECTOR, ".text-body_13px_reg.text-black.font-pretendard")
        for review in review_elements[:10] :
            text = review.text.strip()
            if text :
                all_reviews.append({
                    "상품명": row["상품명"],
                    "리뷰": text
                })
    except Exception as e :
        print("리뷰 수집 실패 :", e)
        continue
        
driver.quit()

wb = Workbook()
ws = wb.active
ws.title = "무신사 리뷰"

ws.append(["상품명", "리뷰"])

for r in all_reviews :
    ws.append([r["상품명"], r["리뷰"]])

wb.save("musinsa_top30_reviews.xlsx")
print("✅ 엑셀파일 저장완료 : musinsa_top30_reviews.xlsx")

1번째 상품리뷰 수집 중 : {'W TWINKLE SELLER LOGO SS TEE WHITE(CV2EMFT517A)'}
리뷰 수집 실패 : Message: no such element: Unable to locate element: {"method":"xpath","selector":"//button[@data-button-id='prd_review_tab']"}
  (Session info: chrome=139.0.7258.128); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#nosuchelementexception
Stacktrace:
0   chromedriver                        0x000000010347b26c cxxbridge1$str$ptr + 2741972
1   chromedriver                        0x00000001034731dc cxxbridge1$str$ptr + 2709060
2   chromedriver                        0x0000000102fbd4fc cxxbridge1$string$len + 90520
3   chromedriver                        0x0000000103004a14 cxxbridge1$string$len + 382640
4   chromedriver                        0x0000000103045db8 cxxbridge1$string$len + 649812
5   chromedriver                        0x0000000102ff8d30 cxxbridge1$string$len + 334284
6   chromedriver                        0x000000010343e66c cxxbr

In [17]:
pip install pymysql

Note: you may need to restart the kernel to use updated packages.


In [18]:
pip install sqlalchemy

Note: you may need to restart the kernel to use updated packages.


In [19]:
import pymysql
import pandas as pd
from sqlalchemy import create_engine

EXCEL_PATH = "musinsa_top30_reviews.xlsx"
DB_NAME = "musinsa_db_v5"
TABLE_NAME = "reviews"
USER = "root"
PASSWORD = ""
HOST = "127.0.0.1"
PORT = 3306

df = pd.read_excel(EXCEL_PATH)
conn = pymysql.connect(host=HOST, user=USER, password=PASSWORD, port=PORT)
cursor = conn.cursor()
cursor.execute(
    f"CREATE DATABASE IF NOT EXISTS {DB_NAME} CHARACTER SET utf8mb4 COLLATE utf8mb4_unicode_ci;"
)
conn.close()

engine_str = f"mysql+pymysql://{USER}:{PASSWORD}@{HOST}:{PORT}/{DB_NAME}?charset=utf8mb4"
engine = create_engine(engine_str)

df.to_sql(name=TABLE_NAME, con=engine, if_exists="replace", index=False)
print(f"✅ 데이터가 MySQL {DB_NAME}의 테이블 {TABLE_NAME}에 성공적으로 저장되었습니다.")

✅ 데이터가 MySQL musinsa_db_v5의 테이블 reviews에 성공적으로 저장되었습니다.
